In [395]:
import tensorflow as tf
import numpy as np

In [396]:
tf.__version__

'1.4.0'

In [397]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [398]:
#vocToInt가 이미 진행된 상태라고 가정
#각각의 정수들은 특정 단어들을 의미 한다.
#샘플데이터의 인풋은 문장이며, 각 문장은 4개의 단어로 구성되어있다.

voca_size = 11
story = [[6,2,3,4],
         [5,2,9,10],
         [4,0,6,7],
         [7,2,3,4],
         [8,2,5,3]]

query = [1,2,3,4]

# query = [[1,2,3,4],
#        [5,2,9,10],
#        [3,0,6,7]]

answer = [10]

# answer = [[5,3,1,2],
#         [9,3,7,2],
#         [6,4,2,2]]

batch_size = len(story)
embedding_size = 6

In [399]:
input_story = tf.placeholder(shape=[batch_size,4],dtype=tf.int32)
input_query = tf.placeholder(shape=[4],dtype=tf.int32)
input_answer = tf.placeholder(shape=[1],dtype=tf.int32)

# input memory

In [400]:
#embedding A matrix
#문장 ={단어1, 단어2, 단어3, ...} -> 하나의 memory vector로 변환해야한다.
#메모리 = sum(A문장) (A는 embedding_matrix)
embeddings_A = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
embedded_X = tf.nn.embedding_lookup(embeddings_A, input_story)

In [401]:
embedded_X

<tf.Tensor 'embedding_lookup:0' shape=(5, 4, 6) dtype=float32>

In [402]:
#메모리로 변환
input_memory = tf.reduce_sum(embedded_X,1)

In [403]:
input_memory

<tf.Tensor 'Sum:0' shape=(5, 6) dtype=float32>

# query

In [404]:
embeddings_B = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1.0, 1.0), dtype=tf.float32)

In [405]:
embedded_Q = tf.nn.embedding_lookup(embeddings_A, query)

In [406]:
embedded_Q

<tf.Tensor 'embedding_lookup_1:0' shape=(4, 6) dtype=float32>

In [407]:
#query -> internal state
internal_state = tf.reduce_sum(embedded_Q,axis=0)

# output memory

In [408]:
embeddings_C = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
embedded_C = tf.nn.embedding_lookup(embeddings_C, input_story)
output_memory = tf.reduce_sum(embedded_C,1)

In [409]:
output_memory

<tf.Tensor 'Sum_2:0' shape=(5, 6) dtype=float32>

In [410]:
tf.stack([output_memory])

<tf.Tensor 'stack:0' shape=(1, 5, 6) dtype=float32>

# build model combining output,input memory and query

In [411]:
# 1. internal_state와 input_memory를 dot product
# 2. 1의 결과에 softmax
# 3. 2의 결과 * output_memory
# 4. Softmax(W*(3의 결과 + internal_State))

W = tf.get_variable(shape=[embedding_size,voca_size],name="W")
p = tf.nn.softmax(tf.tensordot(input_memory,internal_state,axes=1))
p

<tf.Tensor 'Reshape_1:0' shape=<unknown> dtype=float32>

In [412]:
o = tf.matmul(tf.stack([p]),output_memory)
o

<tf.Tensor 'MatMul:0' shape=(?, 6) dtype=float32>

In [413]:
sess.run(tf.global_variables_initializer())

In [414]:
print(sess.run(o,feed_dict={input_story:story,input_query:query,input_answer:answer}))

[[ 0.81834495 -1.13741231  0.42311719 -2.56194639 -1.20918143  0.28953165]]


In [415]:
output = tf.nn.softmax(tf.matmul((internal_state + o),W))
prediction = tf.argmax(output,1)

In [416]:
print(sess.run(output,feed_dict={input_story:story,input_query:query,input_answer:answer}))
print(sess.run(prediction,feed_dict={input_story:story,input_query:query,input_answer:answer}))

[[ 0.06417706  0.01022425  0.05011679  0.10702934  0.01468286  0.06135418
   0.19410521  0.24148077  0.03496668  0.13379157  0.08807131]]
[7]


## 샘플데이터만을 사용했기 때문에 따로 학습하는 부분을 추가하지 않았고 모델만 구현한 예제 입니다.